In [1]:
from pymongo import MongoClient
from faker import Faker
import pprint
import random
import json


Code to generate random sample of data and store in JSON file

In [ ]:
fake = Faker() #initialize Faker object
customers = []
pp = pprint.PrettyPrinter(indent=4)
for i in range(1,25):
    cus = {
        "customer_id": i,
        "name": fake.first_name(),
        "email": fake.email(),
        "address": {
            "street": fake.street_address(),
            "city": fake.city(),
            "state": fake.state()
        }

    }
    customers.append(cus)

with open('customers.json', 'w') as file:
    json.dump(customers,file, indent=4)

In [ ]:
# List of product data as tuples (product_name, category, price)
product_data = [
    ('Laptop', 'Electronics', 1200.00),
    ('Headphones', 'Electronics', 200.00),
    ('Smartphone', 'Electronics', 800.00),
    ('Tablet', 'Electronics', 600.00),
    ('Monitor', 'Electronics', 300.00),
    ('Keyboard', 'Accessories', 50.00),
    ('Mouse', 'Accessories', 30.00),
    ('Desk Chair', 'Furniture', 150.00),
    ('Desk', 'Furniture', 250.00),
    ('Printer', 'Electronics', 400.00),
    ('Earbuds', 'Electronics', 100.00),
    ('Smartwatch', 'Electronics', 250.00),
    ('Bicycle', 'Outdoors', 500.00),
    ('Gaming Console', 'Electronics', 400.00),
    ('Coffee Maker', 'Appliances', 100.00),
    ('Lamp', 'Furniture', 80.00),
    ('TV', 'Electronics', 1500.00),
    ('Refrigerator', 'Appliances', 1000.00),
    ('Blender', 'Appliances', 70.00),
    ('Microwave', 'Appliances', 150.00)
]

# Create a list of dictionaries with product data
products = []
for i, (product_name, category, price) in enumerate(product_data, start=101):
    prod = {
        "product_id": i,
        "product_name": product_name,
        "category": category,
        "price": price
    }
    products.append(prod)

# Save the products list to a JSON file
with open('products.json', 'w') as file:
    json.dump(products, file, indent=4)

In [ ]:
# status categories
status = ["Delivered", "Pending", "Shipped", "In Transit"]
fake = Faker()
orders = []
pp = pprint.PrettyPrinter(indent=4)
for i in range(401,430):
    cus = {
        "order_id": i,
        "customer_id": random.randint(1,25),
        "order_date": faker.date_this_year().strftime('%y-%m-%d'),
        "status": random.choice(status)

    }
    orders.append(cus)

with open('orders.json', 'w') as file:
    json.dump(orders,file, indent=4)

In [ ]:
fake = Faker()
orders_items = []
pp = pprint.PrettyPrinter(indent=4)
for i in range(301,340):
    cus = {
        "order_item_id": i,
        "order_id": random.randint(401,430),
        "product_id": random.randint(101, 121),
        "quantity": random.randint(1,10),
        "price": round(random.uniform(30, 3001),2)

    }
    orders_items.append(cus)

with open('orders_items.json', 'w') as file:
    json.dump(orders_items,file, indent=4)

### Database Setup and Data Insertion

In [19]:
client = MongoClient('mongodb://localhost:27017/')
db = client['E-Commerce']
customers = db['customers']
products = db['products']
orders = db['orders']
orders_items = db['order_items']

In [ ]:
customers.insert_many(cus)
products.insert_many(prod)
orders.insert_many(orders)
orders_items.insert_many(orders_items)

### Analytical Queries

In [20]:
"""
Which product categories generate the highest revenue?
    Use $group to calculate revenue by category.
    Sort the results in descending order.
"""
pipeline1 = [
    {
        "$lookup": {
            'from': "order_items",
            'localField': 'order_id',
            'foreignField': 'order_id',
            'as': 'order_details'
            }
        },
        {
            '$unwind': '$order_details'
        },
        {
            '$lookup': {
                'from': 'products',
                'localField': 'order_details.product_id',
                'foreignField': 'product_id',
                'as': 'product_details'
            },
        },
        {
            '$unwind': '$product_details'
        },
        {
            '$group': {
                '_id': '$product_details.category',
                'total_revenue': {
                    '$sum': {
                        '$multiply': ['$order_details.quantity', '$order_details.price']
                    }
                }
            }
        },
        {
            '$sort': {'total_revenue': -1}
        }   
]
result = orders.aggregate(pipeline1)
for doc in result:
    print(doc)


{'_id': 'Electronics', 'total_revenue': 210480.24}
{'_id': 'Accessories', 'total_revenue': 68956.55}
{'_id': 'Furniture', 'total_revenue': 35245.729999999996}
{'_id': 'Outdoors', 'total_revenue': 27474.0}
{'_id': 'Appliances', 'total_revenue': 12580.55}


In [7]:
"""
Which states have the highest number of customers?
    Use $group to count customers by state.
    Sort the results in descending order.
"""
pipeline2 = [
    {
        '$group': {
            '_id': '$address.state',
            'total_customer': {'$sum': 1}
        }
    },
    {
        '$sort': {'total_customer': -1}
    },
    {'$limit': 3
     }
]

result = customers.aggregate(pipeline2)
for doc in result:
    print(doc)

{'_id': 'Missouri', 'total_customer': 2}
{'_id': 'Ohio', 'total_customer': 2}
{'_id': 'Idaho', 'total_customer': 2}


In [64]:
'''
What are the top 3 most expensive products sold in each order?
Use $lookup and $sort to find the top products in each order.
'''
pipeline3 = [
    {
        '$lookup': {
        'from': 'products',
        'localField': 'product_id',
        'foreignField': 'product_id',
        'as': 'product_order_items'
        }
    },
    {
        "$unwind": "$product_order_items"
    },
    {'$sort': {'product_order_items.price': -1}},
    {
        '$group': {
            '_id': '$order_id',
            'top_products': {
                '$push': {
                    'product_name': '$product_order_items.product_name',
                    'price': '$product_order_items.price'
                }
            }
            }
        },
    {
        '$project': {
            'top_products': {'$slice': ['$top_products',3]}
        }
    }
]

result = orders_items.aggregate(pipeline3)
for re in result:
    print(re)

{'_id': 408, 'top_products': [{'product_name': 'Tablet', 'price': 600}, {'product_name': 'Printer', 'price': 400}]}
{'_id': 429, 'top_products': [{'product_name': 'Coffee Maker', 'price': 100}]}
{'_id': 402, 'top_products': [{'product_name': 'Headphones', 'price': 200}, {'product_name': 'Earbuds', 'price': 100}]}
{'_id': 404, 'top_products': [{'product_name': 'Refrigerator', 'price': 1000}, {'product_name': 'Microwave', 'price': 150}]}
{'_id': 427, 'top_products': [{'product_name': 'Smartphone', 'price': 800}, {'product_name': 'Gaming Console', 'price': 400}]}
{'_id': 428, 'top_products': [{'product_name': 'Monitor', 'price': 300}]}
{'_id': 410, 'top_products': [{'product_name': 'Smartphone', 'price': 800}]}
{'_id': 420, 'top_products': [{'product_name': 'Smartphone', 'price': 800}]}
{'_id': 423, 'top_products': [{'product_name': 'Smartphone', 'price': 800}]}
{'_id': 401, 'top_products': [{'product_name': 'Smartwatch', 'price': 250}, {'product_name': 'Keyboard', 'price': 50}]}
{'_id': 

### Schema Design and Optimization

In [65]:
# create an index of customer_id in Customers collection
customers.create_index(['customer_id'])
# create an indexes of customer_id and order_id in Orders collection
orders.create_index([
    ('order_id',1),
    ('customer_id', 1)
])
# create an indexes of order_id and product_id in Orders_items collection
orders_items.create_index([
    ('order_id', 1),
    ('product_id', 1)
])

'order_id_1_product_id_1'